In [1]:
from dotenv import load_dotenv
import requests
import os
load_dotenv()

MINDSDB_URL = 'http://127.0.0.1:47334/api/sql/query'


In [2]:
def mindsdb_query(url, sql_query):
    headers = {"Content-Type": "application/json"}
    return requests.post(headers=headers, json={"query": sql_query}, url= url)

In [ ]:
def create_db(url:str, payload:dict, headers: dict):
    return requests.post(url=url, json=payload, headers=headers)


In [ ]:
url = "http://127.0.0.1:47334/api/databases"

payload = {"database": {
        "name": "redshift_datasource",
        "engine": "redshift",
        "parameters": {
            "user": os.environ["DEST_USERNAME"],
            "password": os.environ["DEST_PASS"],
            "host": os.environ["DEST_HOST"],
            "port": 5439,
            "database": os.environ["DEST_DB"]
        }
    }}
headers = {"Content-Type": "application/json"}

res = create_db(url=url, payload=payload, headers=headers)

print(res.text)

In [ ]:
API_KEY= os.environ["GEMINI_API_KEY"]

sql_query = f"""
CREATE ML_ENGINE google_gemini_engine
FROM google_gemini
USING
      api_key = {API_KEY};
"""

mindsdb_query(url=MINDSDB_URL, sql_query=sql_query)

In [ ]:
sql_query = f"""
CREATE MODEL flight_assistant
PREDICT answer
USING
    engine = 'google_gemini_engine',
    model_name = 'gemini-2.0-flash',
    mode = 'conversational',
    user_column = 'question',
    assistant_column = 'answer',
    prompt_template = 'You are a specialized assistant for handling flight updates.
    The primary assistant delegates work to you whenever the user needs help updating their bookings.
    Confirm the updated flight details with the customer and inform them of any additional fees.
    When searching, be persistent. Expand your query bounds if the first search returns no results. ';
"""

mindsdb_query(url=MINDSDB_URL, sql_query=sql_query)

In [ ]:
sql_query = f"""
CREATE SKILL flight_assistant_skill
USING
    type = 'text2sql',
    database = 'redshift_datasource',
    tables = ['flights','ticket_flights', 'tickets', 'boarding_passes']
"""
mindsdb_query(url=MINDSDB_URL, sql_query=sql_query)

In [ ]:
sql_query = f"""
CREATE AGENT flight_agent
USING
    model = 'flight_assistant',
    skills = ['flight_assistant_skill'];
"""
mindsdb_query(url=MINDSDB_URL,sql_query=sql_query)

In [ ]:
from datetime import date, datetime
from typing import Optional

import pytz



def fetch_user_flight_info(passenger_id) -> list[dict]: 
    """Fetch all tickets for the user along with corresponding flight information and seat assignments.
    Returns:
      A list of dictionaries where each dictionary contains the ticket details,
      associated flight details, and the seat assignments for each ticket belonging to the user.
    """
    if not passenger_id:
        raise ValueError("No passenger Id given")

    sql_query= f"""
    SELECT
      t.ticket_no, t.book_ref,
        f.flight_id, f.flight_no, f.departure_airport, f.arrival_airport, f.scheduled_departure, f.scheduled_arrival,
        bp.seat_no, tf.fare_conditions
    FROM 
        redshift_datasource.tickets t
        JOIN redshift_datasource.ticket_flights tf ON t.ticket_no = tf.ticket_no
        JOIN redshift_datasource.flights f ON tf.flight_id = f.flight_id
        JOIN redshift_datasource.boarding_passes bp ON bp.ticket_no = t.ticket_no AND bp.flight_id = f.flight_id
    WHERE 
        t.passenger_id = {passenger_id}

    """

    data = mindsdb_query(sql_query)
    return data or []


    
def search_flights( 
            departure_airport: Optional[str]= None,
            arrival_airport: Optional[str] = None,
            start_time: Optional[date | datetime] = None,
            end_time: Optional[date| datetime ] = None,
            limit: int = 20
    )-> list[dict ]:
  
       """ Search for flights based on departure airport, arrival airport, and departure time range"""
       conditions = []

       if departure_airport:
        conditions.append(f"departure_airport = '{departure_airport}'")

       if arrival_airport:
        conditions.append(f"arrival_airport = '{arrival_airport}'")

       if start_time:
        conditions.append(f"scheduled_departure >= '{start_time}'")

       if end_time:
        conditions.append(f"scheduled_departure <= '{end_time}'")

       where_clause = " AND ".join(conditions)
       if where_clause:
        sql_query = f"SELECT * FROM flights WHERE {where_clause} LIMIT {limit}"
       else:
        sql_query = f"SELECT * FROM flights LIMIT {limit}"

       return mindsdb_query(sql_query)

        


def update_ticket_to_new_flight(
          ticket_no: str, new_flight_id: int, passenger_id
)-> str:
     """Update the user's ticket to a new valid flight."""

     if not passenger_id:
        raise ValueError("No passenger ID given.")

     sql_query_1 = f"""
    SELECT scheduled_departure FROM flights WHERE flight_id = {new_flight_id}
    """
     new_flight_data = mindsdb_query(sql_query_1)
     if not new_flight_data:
        return "Invalid new flight ID provided."

     try:
        departure_time_str = new_flight_data[0]["scheduled_departure"]
        departure_time = datetime.strptime(departure_time_str, "%Y-%m-%d %H:%M:%S.%f%z")
     except Exception:
        return "Error parsing scheduled departure time from new flight."

     current_time = datetime.now(tz=pytz.timezone("Etc/GMT-3"))
     time_until = (departure_time - current_time).total_seconds()

     if time_until < 3 * 3600:
        return f"Not permitted to reschedule to a flight within 3 hours. Selected flight is at {departure_time}."

     check_ticket_query = f"""
     SELECT ticket_no FROM tickets WHERE ticket_no = {ticket_no} AND passenger_id = '{passenger_id}'
     """
     ticket_check = mindsdb_query(check_ticket_query)
     if not ticket_check:
        return f"Current signed-in passenger with ID {passenger_id} is not the owner of ticket {ticket_no}."

     update_query = f"""
     UPDATE ticket_flights SET flight_id = {new_flight_id} WHERE ticket_no = {ticket_no}
     """
     response = mindsdb_query(update_query)
     return "Ticket successfully updated to new flight." if response else "Failed to update ticket."

           


def cancel_ticket(ticket_no: str, passenger_id) -> str:
     """Cancel the user's ticket and remove it from the db."""
     if not passenger_id:
        raise ValueError("No passenger ID given.")

     check_ticket_query = f"""
     SELECT ticket_no FROM tickets WHERE ticket_no = {ticket_no} AND passenger_id = '{passenger_id}'
     """
     ticket_check = mindsdb_query(check_ticket_query)
     if not ticket_check:
        return f"Current signed-in passenger with ID {passenger_id} is not the owner of ticket {ticket_no}."

     delete_flight_query = f"""
     DELETE FROM ticket_flights WHERE ticket_no = {ticket_no}
     """
     delete_resp = mindsdb_query(delete_flight_query)
     return "Ticket successfully cancelled." if delete_resp else "Failed to cancel the ticket."


sql_query = f"""
CREATE MODEL flight_assistant
    PREDICT answer
    USING
        engine = 'langchain_engine',
        model_name = 'google_gemini_model',
        provider = 'mindsdb',
        mode = 'conversational',
        user_column = 'question',
        assistant_column = 'answer',
        verbose = 'true',
        prompt_template = "You are a helpful and intelligent customer support assistant for a travel agency, focused on FLIGHT-related issues.
        Your task is to understand the user's questions and resolve them using available tools.

        Whenever you're handling sensitive operations (like cancellations or rebookings), ask the user for confirmation before proceeding.

        When a tool requires parameters, extract those from the user’s input. For example, if the user says 'Show my flights,' extract the passenger ID and call `fetch_user_flight_info` with it.

        Be persistent. If one query returns no results, expand your search before giving up. If you don’t understand something, ask the user for clarification.

        {{question}}";
"""

sql_query_2=f"""
SELECT question,answer
FROM flight_agent
WHERE question = 'Hi there, my passenger_id is 2883 989356.get details of any upcoming flights and cancel the ticket. Tell me what tools you used to get and perform this tasks.'
USING
   tools = [
   'fetch_user_flight_info',
   'search_flights',
   'update_ticket_to_new_flight',
   'cancel_ticket'
   ]
"""
res=mindsdb_query(url=MINDSDB_URL, sql_query=sql_query_2)
print(res.text)
# json_data = json.loads(res.text)
# data = json_data.get("data",[])
# print(data)

{"type": "table", "data": [["Hi there, my passenger_id is 2883 989356.get details of any upcoming flights and cancel the ticket. Tell me what tools you used to get and perform this tasks.", "I found one ticket number '9880005432000994' associated with your passenger ID. However, there are no upcoming flights associated with this ticket. To proceed with the cancellation, I need your confirmation. Also, please note that depending on the ticket conditions, a cancellation fee may apply.\n```"]], "column_names": ["question", "answer"], "context": {"show_secrets": false, "db": "mindsdb"}}


In [3]:
sql_query = f"""
CREATE MODEL car_rental_assistant
PREDICT answer
    USING
        engine = 'langchain_engine',
        model_name = 'google_gemini_model',
        provider = 'mindsdb',
        mode = 'conversational',
        user_column = 'question',
        assistant_column = 'answer',
        verbose = 'true',
        prompt_template = "You are a specialized assistant for handling car rental bookings.
            The primary assistant delegates work to you whenever the user needs help booking a car rental.
            Search for available car rentals based on the user's preferences and confirm the booking details with the customer.
            When searching, be persistent. Expand your query bounds if the first search returns no results.

        {{question}}";
"""

res = mindsdb_query(url=MINDSDB_URL, sql_query=sql_query)
print(res.text)

{"type": "table", "data": [["car_rental_assistant", "langchain", "mindsdb", true, 1, "generating", null, "answer", "up_to_date", "25.4.1.0", null, null, "{'__mdb_sql_task': None, 'target': 'answer', 'using': {'model_name': 'google_gemini_model', 'provider': 'mindsdb', 'mode': 'conversational', 'user_column': 'question', 'assistant_column': 'answer', 'verbose': 'true', 'prompt_template': \"You are a specialized assistant for handling car rental bookings.\\n            The primary assistant delegates work to you whenever the user needs help booking a car rental.\\n            Search for available car rentals based on the user's preferences and confirm the booking details with the customer.\\n            When searching, be persistent. Expand your query bounds if the first search returns no results.\\n\\n        {question}\"}}", null]], "column_names": ["NAME", "ENGINE", "PROJECT", "ACTIVE", "VERSION", "STATUS", "ACCURACY", "PREDICT", "UPDATE_STATUS", "MINDSDB_VERSION", "ERROR", "SELECT_DA

In [4]:
sql_query = f"""

CREATE SKILL car_rental_assistant_skill
USING
    type = 'text2sql',
    database = 'redshift_datasource',
    tables = ['car_rentals']
"""
res = mindsdb_query(url=MINDSDB_URL, sql_query=sql_query)
print(res.text)

{"type": "ok", "context": {"show_secrets": false, "db": "mindsdb"}}


In [5]:
sql_query = f"""

CREATE AGENT car_rental_agent
USING
    model = 'car_rental_assistant',
    skills = ['car_rental_assistant_skill'];
"""
res = mindsdb_query(url=MINDSDB_URL, sql_query=sql_query)
print(res.text)

{"type": "ok", "context": {"show_secrets": false, "db": "mindsdb"}}
